# MCP Server Deployment in Bedrock AgentCore Runtime

This workshop demonstrates how to deploy and use Model Context Protocol (MCP) servers with Amazon Bedrock AgentCore Runtime, enabling scalable and secure deployment of custom tools for AI agents.

## Overview

In this lab, you will:
- Create a custom MCP server with web search functionality
- Set up authentication using Amazon Cognito
- Deploy the MCP server to Bedrock AgentCore Runtime
- Test the deployed server with Strands Agents

## Prerequisites

Before starting this lab, ensure you have:
- AWS credentials configured (IAM role or environment variables)
- Required Python packages installed
- Basic understanding of MCP and Bedrock AgentCore concepts

If you're not running in an environment with an IAM role assumed, set your AWS credentials as environment variables:

In [ ]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

Install required packages for MCP server development, Strands Agents and Bedrock AgentCore SDK:

In [ ]:
#%pip install -q ddgs mcp strands-agents strands-agents-tools bedrock-agentcore bedrock-agentcore-starter-toolkit rich

## What is Bedrock AgentCore Runtime for MCP?

Amazon Bedrock AgentCore Runtime allows you to deploy Model Context Protocol (MCP) servers as managed, scalable services. Key benefits include:

- **Scalability**: Automatically scales based on demand
- **Security**: Built-in authentication and authorization
- **Managed Infrastructure**: No need to manage servers or containers
- **Integration**: Seamless integration with Bedrock services

MCP servers provide tools and resources that AI agents can use to extend their capabilities, such as web search, database access, or custom business logic.

### Creating a Custom MCP Server

Let's create a simple MCP server that provides web search functionality using DuckDuckGo. This server will be deployed to AgentCore Runtime for scalable use.

In [ ]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

# Define a websearch tool
@mcp.tool()
def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> list:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

In [ ]:
%%writefile requirements.txt
ddgs
mcp
bedrock-agentcore

### Local Testing of MCP Server (Optional - skip if port 8000 is occupied in workshop environment)

Before deploying to AgentCore Runtime, test the MCP server locally.

### Step 1: Start the MCP Server

Run the MCP server in a terminal:

```bash
cd 04-bedrock-agentcore-runtime-mcp/
uv pip install -r requirements.txt
uv run mcp_server.py
```
or
```bash
cd 04-bedrock-agentcore-runtime-mcp/
pip install -r requirements.txt
python mcp_server.py
```

### Step 2: Test with Strands Agent

Execute the following code to test the integration:

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client

# Connect to the web search MCP server
print("\nConnecting to MCP Server...")
mcp_url = f"http://localhost:8000/mcp"
websearch_server = MCPClient(lambda: streamablehttp_client(mcp_url))

with websearch_server:
    mcp_tools = (websearch_server.list_tools_sync())
    print(f"Available MCP tools: {[tool.tool_name for tool in mcp_tools]}")

    # Create agent with self-built MCP tools
    agent = Agent(
        model=BedrockModel(model_id="us.amazon.nova-pro-v1:0"),
        system_prompt = """You are a helpful assistant that provides concise responses.
                        If web search, always use websearch tool first.
                        """,
        tools=mcp_tools,
    )

    agent("What is Bedrock AgentCore?")

### Stop the MCP server running locally

After test MCP server locally, press `Ctrl+C` in terminal to stop the MCP server running locally.

## Deploy MCP Server in Bedrock AgentCore Runtime with Authentication 
Now we'll configure and deploy our MCP server to Bedrock AgentCore Runtime. This process involves creating dependencies, configuring authentication, and deploying the service.
![bedrock-agentcore-runtime-launch](images/runtime-launch.png)

### Step 1: Setting up Amazon Cognito for Inbound Authentication

Create a Cognito User Pool for secure access to the deployed MCP server.

Components Created
- **User Pool**: Manages user identities
- **App Client**: Enables application authentication
- **Test User**: For testing the authentication flow

In [ ]:
import boto3

region = boto3.session.Session().region_name

# Initialize Cognito client
cognito_client = boto3.client('cognito-idp', region_name=region)

# Create User Pool
user_pool_response = cognito_client.create_user_pool(
    PoolName='MCPServerPool',
    Policies={
        'PasswordPolicy': {
            'MinimumLength': 8
        }
    }
)
cognito_pool_id = user_pool_response['UserPool']['Id']

# Create App Client
app_client_response = cognito_client.create_user_pool_client(
    UserPoolId=cognito_pool_id,
    ClientName='MCPServerPoolClient',
    GenerateSecret=False,
    ExplicitAuthFlows=[
        'ALLOW_USER_PASSWORD_AUTH',
        'ALLOW_REFRESH_TOKEN_AUTH'
    ]
)
cognito_client_id = app_client_response['UserPoolClient']['ClientId']

# Create User
cognito_client.admin_create_user(
    UserPoolId=cognito_pool_id,
    Username='testuser',
    TemporaryPassword='Temp123!',
    MessageAction='SUPPRESS'
)

# Set Permanent Password
cognito_client.admin_set_user_password(
    UserPoolId=cognito_pool_id,
    Username='testuser',
    Password='MyPassword123!',
    Permanent=True
)

# Output the required values
print(f"Pool id: {cognito_pool_id}")
print(f"Discovery URL: https://cognito-idp.{region}.amazonaws.com/{cognito_pool_id}/.well-known/openid-configuration")
print(f"Client ID: {cognito_client_id}")

### Step 2: Configuring Bedrock AgentCore Runtime

Set up the Bedrock AgentCore Runtime configuration with automatic resource creation.

**Generated Artifacts:**
This step creates essential deployment files:
- **Dockerfile**: Container configuration for the MCP Server
- **.dockerignore**: list the excluded files when docker build
- **.bedrock_agentcore.yaml**: Runtime deployment configuration

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
import boto3

region = boto3.session.Session().region_name
print(f"Using AWS region: {region}")

agentcore_runtime = Runtime()

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    protocol="MCP",
    agent_name="mcp_server_agentcore",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_client_id],
            "discoveryUrl": f"https://cognito-idp.{region}.amazonaws.com/{cognito_pool_id}/.well-known/openid-configuration",
        }
    }
)
print("Configuration completed ✓")

### Step 3: Deploying to Bedrock AgentCore Runtime

Launch the deployment process using AWS CodeBuild for containerization and deployment.

**Deployment Process:**
- Builds a containerized version of your MCP Server
- Creates required AWS resources (ECR repository, IAM roles)
- Push container image to Amazon ECR
- Deploys to AgentCore Runtime as a managed, auto-scaling service

In [ ]:
launch_result = agentcore_runtime.launch()

### Verify Deployment Status

Check the deployment status and wait for the runtime to be ready:

In [ ]:
import time

print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime is READY!")
else:
    print(f"⚠ AgentCore Runtime status: {status}")
    
print(f"Final status: {status}")

mcp_runtime_id = launch_result.agent_id
mcp_runtime_arn = launch_result.agent_arn
ecr_repo_name = launch_result.ecr_uri.split('/')[1]
codebuild_name = launch_result.codebuild_id.split(':')[0]
print(f"MCP AgentCore Runtime ID: {mcp_runtime_id}")
print(f"MCP AgentCore Runtime ARN: {mcp_runtime_arn}")
print(f"ECR Repo for MCP AgentCore Runtime: {ecr_repo_name}")
print(f"CodeBuild Project for Strands AgentCore Runtime: {codebuild_name}")

### Testing the Deployed MCP Server as tool with Strands Agent

Now let's test our deployed MCP server by connecting to it through the Bedrock AgentCore Runtime endpoint with proper authentication.

First, we obtain the access token from Cognito authentication with username and password.

In [ ]:
import boto3

region = boto3.session.Session().region_name

# Get bearer token (access token) from Cognito Auth 
cognito_client = boto3.client('cognito-idp', region_name=boto3.session.Session().region_name)
auth_response = cognito_client.initiate_auth(
    ClientId=cognito_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME': 'testuser',
        'PASSWORD': 'MyPassword123!'
    }
)
bearer_token = auth_response['AuthenticationResult']['AccessToken']
print(bearer_token)

Then, we set the access token as bearer in the request header to securely connect MCP server hosting in AgentCore Runtime.

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client

region = boto3.session.Session().region_name

mcp_runtime_arn = launch_result.agent_arn
encoded_arn = mcp_runtime_arn.replace(':', '%3A').replace('/', '%2F')

# Connect to the Web Search MCP server
print("\nConnecting to MCP Server...")
mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
headers = {
    "Authorization": f"Bearer {bearer_token}",
    #"Content-Type": "application/json"
}
websearch_server = MCPClient(lambda: streamablehttp_client(mcp_url, headers))

with websearch_server:
    mcp_tools = (websearch_server.list_tools_sync())
    print(f"Available tools: {[tool.tool_name for tool in mcp_tools]}")

    # Create agent with self-built MCP tools
    agent = Agent(
        model=BedrockModel(model_id="us.amazon.nova-pro-v1:0"),
        system_prompt = """You are a helpful assistant that provides concise responses.
                        If web search, always use websearch tool first.
                        """,
        tools=mcp_tools,
    )

    agent("What is Bedrock AgentCore?")

Let's examine the detailed execution flow of the agent loop to understand how the agent processes requests and generates responses:

In [ ]:
from rich.table import Table
import rich
import json

console = rich.get_console()

console.print("Agent Loop Detail")
console.rule()
console.print(f"Number of Loops: {agent.event_loop_metrics.cycle_count}")

table = Table(title="Agent Messages", show_lines=True)
table.add_column("Role", style="green")
table.add_column("Text", style="magenta")
table.add_column("Tool Name", style="cyan")
table.add_column("Tool Input", style="cyan")
table.add_column("Tool Result", style="cyan")

for message in agent.messages:
    text = [content["text"] for content in message["content"] if "text" in content]
    tool_name = [content["toolUse"]["name"] for content in message["content"] if "toolUse" in content]
    tool_input = [content["toolUse"]["input"] for content in message["content"] if "toolUse" in content]
    tool_result = [content["toolResult"]["content"][0] for content in message["content"] if "toolResult" in content]
    table.add_row(message["role"], text[-1] if text else "", 
                  tool_name[-1] if tool_name else "", 
                  json.dumps(tool_input[-1], indent=2) if tool_input else "", 
                  (json.dumps(tool_result[-1], indent=2)[:500]+"\n.\n.\n." if len(str(tool_result[-1])) > 500 else json.dumps(tool_result[-1], indent=2)) if tool_result else "")

console.print(table)

## Resource Cleanup (Optional)

Clean up the deployed resources:

In [ ]:
import boto3
import os

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr',region_name=region)
codebuild_client = boto3.client('codebuild',region_name=region)
cognito_client = boto3.client('cognito-idp', region_name=region)

try:
    print("Deleting AgentCore Runtime...")
    agentcore_control_client.delete_agent_runtime(agentRuntimeId=mcp_runtime_id)
    print("✓ AgentCore Runtime deletion initiated")

    print("Deleting ECR repository...")
    ecr_client.delete_repository(repositoryName=ecr_repo_name, force=True)
    print("✓ ECR repository deleted")

    print("Deleting CodeBuild Project...")
    codebuild_client.delete_project(name=codebuild_name)
    print("✓ CodeBuild Project deleted")

    print("Deleting Cognito User Pool...")
    cognito_client.delete_user_pool(UserPoolId=cognito_pool_id)
    print("✓ Cognito User Pool deleted")

    print("Deleting Bedrock AgentCore configuration file...")
    os.remove(".bedrock_agentcore.yaml") 
    print("✓ .bedrock_agentcore.yaml deleted")
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")

## Conclusion

In this lab, you successfully:

- ✅ Created a custom MCP server with web search functionality using DuckDuckGo
- ✅ Set up Amazon Cognito for secure authentication to the MCP server
- ✅ Configured and deployed the MCP server to Bedrock AgentCore Runtime
- ✅ Integrated the deployed MCP server with Strands Agents for AI-powered workflows
  
## Key Benefits of AgentCore Runtime for MCP

- **Scalable Deployment**: Managed infrastructure for MCP servers without server management
- **Secure Authentication**: Built-in support for Cognito and other authentication methods
- **Easy Integration**: Seamless connection with Strands Agents and other AI frameworks
- **Production Ready**: Enterprise-grade reliability and monitoring capabilities